<p style="text-align:center;font-size:32px;">DATASET</p>


In [8]:
import numpy as np
import pandas as pd
import re
import requests
import time

from bs4 import BeautifulSoup
from PB_funciones import nuevo_libro, titulo_rect, sales_rect, serie_extr

ModuleNotFoundError: No module named 'PB_funciones'

<p style="text-align:center;font-size:20px;color:blue;">---------- DATAFRAMES DE REFERENCIA ----------</p>

In [ ]:
# DATAFRAME DE LIBROS MÁS VENDIDOS-----------------------------------------------------------------------------------------------

# Web scraping de la página de Wikipedia con los libros más vendidos (inglés)
url = "https://en.wikipedia.org/wiki/List_of_best-selling_books"
respuesta = requests.get(url)
tablas_libros = pd.read_html(respuesta.content, encoding = "utf8")

# Creación del DataFrame
df_bsbooks = pd.DataFrame()
df1 = pd.DataFrame(tablas_libros[0])
df2 = pd.DataFrame(tablas_libros[1])
df3 = pd.DataFrame(tablas_libros[2])
df4 = pd.DataFrame(tablas_libros[3])
df_bsbooks = pd.concat([df1,df2,df3,df4], axis = 0)


In [ ]:
# Se resetea el valor del índice
df_bsbooks.reset_index(inplace = True)

# Se combinan las columnas de ventas aproximadas
df_bsbooks.loc[df_bsbooks["Approximate sales"].isnull(),"Approximate sales"] = df_bsbooks["Approx. sales"]

# Se elimina la columna de Approx. sales
df_bsbooks.drop(["index","Approx. sales"], axis = 1, inplace = True)

# Se eliminan los títulos originales de la columna título.
for index, elem in enumerate(df_bsbooks["Book"]):
    df_bsbooks.iloc[index,0] = titulo_rect(elem)

# Se eliminan los corchetes y paréntesis de la columna sales.
simbolos = ["[", "("]
for index, elem in enumerate(df_bsbooks["Approximate sales"]):     
    df_bsbooks.iloc[index,4] = sales_rect(elem, simbolos)
    
# Se renombran y ordenan las columnas
df_bsbooks.rename(columns = {"Book":"Título","Author(s)":"Autor","Genre":"Género","First published":"Año de publicación","Approximate sales":"Ventas","Original language":"Idioma original"}, inplace = True)


In [ ]:
# Se guarda el dataframe en formato csv
df_bsbooks.to_csv("./data/BSBooks.csv", index = False)
df_bsbooks


,Título,Autor,Idioma original,Año de publicación,Ventas,Género
0,A TALE OF TWO CITIES,Charles Dickens,English,1859,>200 million,Historical fiction
1,THE LITTLE PRINCE,Antoine de Saint-Exupéry,French,1943,200 million,"Fantasy, children's fiction"
2,THE ALCHEMIST,Paulo Coelho,Portuguese,1988,150 million,Fantasy
3,HARRY POTTER AND THE PHILOSOPHER'S STONE,J. K. Rowling,English,1997,120 million,"Fantasy, children's fiction"
4,AND THEN THERE WERE NONE,Agatha Christie,English,1939,100 million,Mystery
...,...,...,...,...,...,...
160,FAHRENHEIT 451,Ray Bradbury,English,1953,10 million,NaN
161,ANGELA'S ASHES,Frank McCourt,English,1996,10 million,NaN
162,THE STORY OF MY EXPERIMENTS WITH TRUTH,Mohandas Karamchand Gandhi,Gujarati,1925–1929,10 million,NaN
163,BRIDGET JONES'S DIARY,Helen Fielding,English,1996,10 million,NaN


In [ ]:
# DATAFRAME DE LIBROS MEJOR VALORADOS (GOODREADS)-----------------------------------------------------------------------

# Se lee el archivo .csv y se crea el DataFrame
df_gr = pd.read_csv("./data/Goodreads_bs.csv")


In [ ]:
# Se eliminan los valores nulos
df_gr.dropna(axis = 0, inplace = True)

# Se renombran las columnas
df_gr.rename(columns = {"bookTitle":"Título", "authorName":"Autor", "average_rating":"Valoración", "num_ratings":"Votos"}, inplace = True)

# Se corrige el tipo de la columna Votos
df_gr.Votos.value_counts()
df_gr["Votos"] = df_gr["Votos"].astype(str)
df_gr["Votos"] = df_gr["Votos"].str.replace(",","")
df_gr["Votos"] = df_gr["Votos"].astype(int)

# Se extraen los libros con nota por encima de 4 y al menos 1000 votos para formar un nuevo DataFrame
df_grf = df_gr.loc[(df_gr["Votos"] > 10000) & (df_gr["Valoración"] > 4.0)]
df_grbooks = df_grf.sort_values("Valoración", ascending = False)

# Se resetea el índice
df_grbooks.reset_index(inplace = True)
df_grbooks.drop(columns = "index", inplace = True)

# Se extrae de la columna Título la serie a la que pertenece
df_grbooks["Serie"] = None
for index, elem in enumerate(df_grbooks["Título"]):
    df_grbooks.iloc[index,-1] = serie_extr(elem)

# Se reordenan las columnas
df_grbooks = df_grbooks.reindex(["Título","Autor","Serie","Valoración","Votos"], axis = 1)

# Se elimina la serie de la columna título.
for index, elem in enumerate(df_grbooks["Título"]):
    df_grbooks.iloc[index,0] = titulo_rect(elem)


In [ ]:
# Se guarda el dataframe en formato csv
df_grbooks.to_csv("./data/GRBooks.csv", index = False)
df_grbooks


,Título,Autor,Serie,Valoración,Votos
0,THE COMPLETE CALVIN AND HOBBES,Bill Watterson,None,4.81,40293
1,ALL THE YOUNG DUDES - VOLUME TWO: YEARS 5 - 7,MsKingBean89,All the Young Dudes #2,4.81,17855
2,HEAVEN OFFICIAL'S BLESSING: TIAN GUAN CI FU,Mò Xiāng Tóng Xiù,Novel Vol. 4,4.78,13124
3,MARK OF THE LION TRILOGY,Francine Rivers,None,4.78,13310
4,ALL THE YOUNG DUDES,MsKingBean89,None,4.78,30154
...,...,...,...,...,...
4659,THE GRAPES OF WRATH,John Steinbeck,None,4.01,908458
4660,THE KINDLY ONES,Jonathan Littell,None,4.01,13618
4661,THE ADVENTURES OF CAPTAIN UNDERPANTS,Dav Pilkey,Captain Underpants #1,4.01,53146
4662,MONA LISA OVERDRIVE,William Gibson,Sprawl #3,4.01,44179


In [ ]:
# DATAFRAME DE LIBROS RECOMENDADOS (The Library)--------------------------------------------------------

# Web scraping de la página de The Library
url = "https://www.oclc.org/en/worldcat/library100/top500.html"
respuesta = requests.get(url)
tablas_libros = pd.read_html(respuesta.content, encoding = "utf8")

# Se crea el DataFrame
df_recbooks = pd.DataFrame(tablas_libros[0])


In [ ]:
# Se pone el título en mayúsculas
for index, elem in enumerate(df_recbooks["Title"]):
    df_recbooks.iloc[index,1] = titulo_rect(elem)

# Se renombran las columnas y se elminia la columna Rank
df_recbooks.rename(columns = {"Title":"Título","Author":"Autor"}, inplace = True)
df_recbooks.drop(columns = ["Rank"], axis = 1, inplace = True)


In [ ]:
# Se guarda el dataframe en formato csv
df_recbooks.to_csv("./data/RECBooks.csv", index = False)
df_recbooks


,Título,Autor
0,DON QUIXOTE,Miguel de Cervantes
1,ALICE'S ADVENTURES IN WONDERLAND,Lewis Carroll
2,THE ADVENTURES OF HUCKLEBERRY FINN,Mark Twain
3,THE ADVENTURES OF TOM SAWYER,Mark Twain
4,TREASURE ISLAND,Robert Louis Stevenson
...,...,...
495,STRANGER IN A STRANGE LAND,Robert A. Heinlein
496,VISION IN WHITE,Nora Roberts
497,THE WHIPPING BOY,Sid Fleischman
498,ROOM,Emma Donoghue


In [ ]:
# DATAFRAME DE MEJORES LIBROS SIGLO XXI---------------------------------------------------------------------

# Se lee el archivo y se crea un DataFrame
df_21bbooks = pd.read_csv("./data/Best_Book_21st.csv")


In [ ]:
# Se eliminan varias columnas que no se emplearán en el análisis
df_21bbooks.drop(["id","book_link"], axis = 1, inplace = True)

# Se sustituyen los NaN de las columnas "series", "publisher" y "num_of_page"
df_21bbooks["series"] = df_21bbooks["series"].fillna("NPNS")
df_21bbooks["publisher"] = df_21bbooks["publisher"].fillna("Sin datos")
df_21bbooks["num_of_page"] = df_21bbooks["num_of_page"].fillna(0)

# Se crea una nueva columna Editorial con el formato limpio de los valores anteriores
for indice, elem in enumerate(df_21bbooks["publisher"]):    
    elem = elem.replace("\n","")
    elem = elem.replace(",","")
    elem = elem.strip()
    df_21bbooks.loc[indice,"Editor"] = elem    

# Se crea una nueva columna Serie con el formato limpio de los valores anteriores de serie
for indice, elem in enumerate(df_21bbooks["series"]):    
    elem = elem.replace("(","")
    elem = elem.replace(")","")
    elem = elem.strip()
    df_21bbooks.loc[indice,"Serie"] = elem

# Se extrae del títuo la serie a la que pertenece y se añade a la columna Serie
indcol = df_21bbooks.columns.get_loc("Serie")
for index, elem in enumerate(df_21bbooks["title"]):
    if df_21bbooks.iloc[index,indcol] == "NPNS":
        df_21bbooks.iloc[index,indcol] = serie_extr(elem)

# Se elimina en la columna title la serie a la que pertenece el título
indcol = df_21bbooks.columns.get_loc("title")
for index, elem in enumerate(df_21bbooks["title"]):
    df_21bbooks.iloc[index,indcol] = titulo_rect(elem)
    
# Se corrigen los valores de la columna date_published
df_21bbooks.loc[(df_21bbooks["date_published"] == ",") | (df_21bbooks["date_published"] == ", , , , ,") | (df_21bbooks["date_published"].isna())] = None

# Se convierte la columna fecha en Date y se añade el año a una nueva columna
df_21bbooks["Fecha de publicación"] = pd.to_datetime(df_21bbooks["date_published"], format='mixed')
df_21bbooks["Año de publicación"] = pd.DatetimeIndex(df_21bbooks["Fecha de publicación"]).year

# Se resetea el índice
df_21bbooks.reset_index(inplace = True)

# Se eliminan las columnas duplicadas
df_21bbooks.drop(["index","series","publisher","date_published"], axis = 1, inplace = True)

# Se renombran y ordenan las columnas
df_21bbooks.rename(columns = {"title":"Título","author":"Autor","genre":"Género","lang":"Idioma original","award":"Premios","num_of_page":"Páginas","rate":"Valoración","rating_count":"Votos","review_count":"Reseñas"}, inplace = True)
df_21bbooks = df_21bbooks.reindex(["Título","Autor","Serie","Género","Año de publicación","Premios","Editor","Páginas","Valoración","Votos","Reseñas","Fecha de publicación"], axis = 1)

# Se convierten las columnas Páginas y Año en int
df_21bbooks["Páginas"] = df_21bbooks["Páginas"].astype("Int64")
df_21bbooks["Año de publicación"] = df_21bbooks["Año de publicación"].astype("Int64")


In [ ]:
# Se guarda el dataframe en formato csv
df_21bbooks.to_csv("./data/21BBooks.csv", index = False)
df_21bbooks


,Título,Autor,Serie,Género,Año de publicación,Premios,Editor,Páginas,Valoración,Votos,Reseñas,Fecha de publicación
0,THE MARTIAN,Andy Weir,The Martian #1,"Science Fiction,Fiction,Audiobook,Adventure,Sp...",2014,Kurd-Laßwitz-Preis for Bestes ausländisches We...,Crown,384,4.40,"919,781","77,156",2014-02-11
1,UNDER THE BANNER OF HEAVEN: A STORY OF VIOLENT...,Jon Krakauer,None,"Nonfiction,Religion,History,Crime,True Crime,M...",2004,Colorado Book Award for General Nonfiction (2004),Pan MacMillan,400,3.98,"173,895","10,981",2004-01-01
2,CUTTING FOR STONE,Abraham Verghese,None,"Fiction,Historical,Historical Fiction,Cultural...",2009,"PEN/Hemingway Foundation Award Nominee (2010),...",Alfred A. Knopf,541,4.29,"353,365","26,253",2009-02-03
3,WE NEED TO TALK ABOUT KEVIN,Lionel Shriver,None,"Fiction,Contemporary,Thriller,Horror,Mystery,C...",2006,Orange Prize for Fiction (2005),Harper Perennial,400,4.04,"187,639","14,787",2006-07-03
4,THE IMMORTAL LIFE OF HENRIETTA LACKS,Rebecca Skloot,None,"Nonfiction,Science,History,Biography,Health,Me...",2010,Ambassador Book Award for American Studies (20...,Crown Publishing Group,370,4.07,"646,155","36,574",2010-02-02
...,...,...,...,...,...,...,...,...,...,...,...,...
10013,EDGE OF PASSION,Tina Folsom,Stealth Guardians #1,"Fantasy,Paranormal,Romance,Romance,Paranormal ...",2012,NaN,Sin datos,0,4.01,"1,011",102,2012-03-07
10014,TALES FROM A NOT-SO-POPULAR PARTY GIRL,Rachel Renée Russell,Dork Diaries #2,"Childrens,Middle Grade,Realistic Fiction,Child...",2010,NaN,Aladdin,288,4.24,"54,285","1,480",2010-06-08
10015,BOTTOMS,Tanja Kirschner,None,NaN,2005,NaN,NorthSouth,24,3.55,11,5,2005-09-01
10016,THE WEDDING DRESS,Gary E. Parker,None,NaN,2002,NaN,Victor,123,3.61,23,2,2002-01-09


<p style="text-align:center;font-size:20px;color:blue;">---------- DATAFRAME FINAL ----------</p>

In [ ]:
# Se inicia un DataFrame con los títulos de todas las listas, y se eliminan líneas vacías y duplicados.
df_titulos = pd.DataFrame()
df_titulos["Title"] = pd.concat([df_21bbooks["Título"],df_recbooks["Título"],df_grbooks["Título"],df_bsbooks["Título"]], axis = 0)
df_titulos.dropna(axis = 0, inplace = True)
df_titulos.drop_duplicates(keep = "first", inplace = True)


In [ ]:
# Se crea el DataFrame final que contendrá todos los datos, se añaden los títulos y se resetea el índice
df_databooks = pd.DataFrame()
df_databooks["Título"] = df_titulos.Title.sort_values()
df_databooks["Título"] = df_databooks["Título"].str.encode('latin-1', 'ignore').str.decode('latin-1')
df_databooks.reset_index(inplace = True)
df_databooks.drop(columns = ["index"], axis = 1, inplace = True)


In [ ]:
# Se añaden los datos del DataFrame de mejores libros del siglo XXI
df_databooks = pd.merge(df_databooks,df_21bbooks, on = "Título")

# Se crea la columna premiado para identificar los títulos que han recibido algún premio
df_databooks["Premiado"] = df_databooks["Premios"].notnull()


In [ ]:
# Se añaden los datos del DataFrame de libros recomendados
df_databooks = pd.merge(df_databooks,df_recbooks, on = "Título", how = "left")

# Se crea la columna recomendado para identificar los títulos que se han recomendado
df_databooks["Recomendado"] = df_databooks["Autor_y"].notnull()

# Se eliminan las columnas duplicadas y se renombran las coincidentes
df_databooks.drop("Autor_y", axis = 1, inplace = True)
df_databooks.rename(columns = {"Autor_x":"Autor"}, inplace = True)


In [ ]:
# Se añaden los datos del DataFrame de libros mejor valorados
df_databooks = pd.merge(df_databooks,df_grbooks, on = "Título", how = "left")

# Se crea la columna mejor valorado para identificar los libros mejor valorados
#df_databooks["Mejor Valorado"] = df_databooks["Autor_y"].notnull()
df_databooks["Mejor Valorado"] = False
df_databooks.loc[(df_databooks["Autor_y"].notnull()) | (df_databooks["Valoración_x"] > 4), "Mejor Valorado"] = True

# Se completa la columna serie con los datos aportados por este DataFrame
df_databooks["Serie_y"] = df_databooks["Serie_y"].astype(str)
for i in range(len(df_databooks["Serie_x"])):
    if df_databooks.iloc[i,2] == None:
       if (df_databooks.iloc[i,15] != None)&(df_databooks.iloc[i,15] != "nan"):
           df_databooks.iloc[i,2] = df_databooks.iloc[i,15]   

# Se renombran las columnas coincidentes
df_databooks.rename(columns = {"Autor_x":"Autor","Serie_x":"Serie","Valoración_x":"Valoración","Votos_x":"Votos"}, inplace = True)

# Se eliminan las columnas coincidentes
df_databooks.drop(["Autor_y","Serie_y","Valoración_y","Votos_y"], axis = 1, inplace = True)
        

In [ ]:
# Se añaden los datos del DataFrame de libros más vendidos
df_databooks = pd.merge(df_databooks,df_bsbooks, on = "Título", how = "left")

# Se crea la columna bestseller para identificar los libros mejor valorados
df_databooks["BestSeller"] = df_databooks["Autor_y"].notnull()
df_databooks["BestSeller"].value_counts(dropna = False)

# Se renombran las columnas coincidentes
df_databooks.rename(columns = {"Autor_x":"Autor","Género_x":"Género","Año de publicación_x":"Año de publicación"}, inplace = True)

# Se eliminan las columnas coincidentes
df_databooks.drop(["Autor_y","Año de publicación_y","Género_y"], axis = 1, inplace = True)


In [ ]:
# Se eliminan los valores duplicados según el título
df_databooks = df_databooks.drop_duplicates(subset="Título", keep="first")

# Se eliminan las columnas ventas e idioma original por falta de datos.
df_databooks.drop(["Ventas","Idioma original"], axis = 1, inplace = True)

# Se reordenan las columnas
df_databooks = df_databooks.reindex(["Título","Serie","Autor","Género","Año de publicación","Editor",\
                                     "Páginas","BestSeller","Premiado","Mejor Valorado","Recomendado",\
                                     "Premios","Valoración","Votos","Reseñas","Fecha de publicación"], axis = 1)

# Se asignan valores a los nulos
df_databooks["Serie"] = df_databooks["Serie"].fillna("NPNS")
df_databooks["Género"] = df_databooks["Género"].fillna("Sin datos")
df_databooks["Premios"] = df_databooks["Premios"].fillna("No premiado")

for i, elem in enumerate(df_databooks["Título"]):
    if df_databooks.iloc[i,1] == "None":
        df_databooks.iloc[i,1] = "NPNS"
        
# Se crea una nueva columna que contanga un booleano para los títulos que pertenecen a una serie
df_databooks["Seriado"] = False
df_databooks.loc[(df_databooks["Serie"] != "NPNS"),"Seriado"] = True


In [ ]:
# Se muestra la información
df_databooks.info()


<class 'pandas.core.frame.DataFrame'>
Index: 8751 entries, 0 to 9902
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Título                8751 non-null   object        
 1   Serie                 8751 non-null   object        
 2   Autor                 8751 non-null   object        
 3   Género                8751 non-null   object        
 4   Año de publicación    8751 non-null   Int64         
 5   Editor                8751 non-null   object        
 6   Páginas               8751 non-null   Int64         
 7   BestSeller            8751 non-null   bool          
 8   Premiado              8751 non-null   bool          
 9   Mejor Valorado        8751 non-null   bool          
 10  Recomendado           8751 non-null   bool          
 11  Premios               8751 non-null   object        
 12  Valoración            8751 non-null   float64       
 13  Votos                 8

In [ ]:
# Se guarda el dataframe en formato csv
df_databooks.to_csv("./data/DataBooks.csv", index = False)
